### Preprocessing Data

In [1]:
%matplotlib nbAgg
%matplotlib nbAgg
#>>>>>> Imports <<<<<<
from sklearn import preprocessing
import xarray as xr
import numpy
import pickle

#>>>>>> Variables <<<<<<
# Step 1: Preparing data
input_dir = "/mount/studenten/arbeitsdaten-studenten1/alatrarm/DL-project/" 
#names = ['data_prosody.train', 'data_logMel.train', 'data_prosody.valid', 'data_logMel.valid', 'data_prosody.test', 
#         'data_logMel.test']
file_paths = [input_dir + "data_prosody.train", input_dir + "data_logMel.train", input_dir + "data_prosody.valid", 
              input_dir + "data_logMel.valid", input_dir + "data_prosody.test", input_dir + "data_logMel.test"]

logMel_train_ids = []
logMel_train_features = []
logMel_train_labels = []
logMel_valid_ids = []
logMel_valid_features = []
logMel_valid_labels = []
logMel_test_ids = []
logMel_test_features = []
logMel_test_labels = []
input_height = 750 # height = no. of rows/segments (750)
input_width = 26 # this is the no. of cols (26 in logmel file)
# Fix random seed for reproducibility
seed = 42
numpy.random.seed(seed)

#>>>>>> functions <<<<<<
def unpickle_data(path):
    '''returns 3 lists: data Ids, features, labels'''    
    with open(path) as f_in:
        ids = pickle.load(f_in)
        features = pickle.load(f_in)
        labels = pickle.load(f_in)
        #print(len(ids), len(features), len(labels))
        #print("{0}\n{1}\n{2}".format(ids[10],features[10],labels[10]))
        return ids, features, labels

# 1.1 unpickle files
logMel_file_paths = list(x for x in file_paths if "logMel" in x)
#print(logMel_file_paths)
logMel_train_ids,logMel_train_features,logMel_train_labels = unpickle_data(logMel_file_paths[0])
logMel_valid_ids,logMel_valid_features,logMel_valid_labels = unpickle_data(logMel_file_paths[1])
logMel_test_ids,logMel_test_features,logMel_test_labels = unpickle_data(logMel_file_paths[2])

#1.2 Normalize Data
train_scaler = preprocessing.Normalizer(norm='l2', copy=False).fit(logMel_train_features[0,:,:])
valid_scaler = preprocessing.Normalizer(norm='l2', copy=False).fit(logMel_valid_features[0,:,:])
test_scaler = preprocessing.Normalizer(norm='l2', copy=False).fit(logMel_test_features[0,:,:])
train_scaler.transform(logMel_train_features[0,:,:])
valid_scaler.transform(logMel_valid_features[0,:,:])
test_scaler.transform(logMel_test_features[0,:,:])
#print(logMel_train_features[0])

In [2]:
# 1.3.0 encode the string labels into numeric values    
le = preprocessing.LabelEncoder()
le.fit(["angry", "happy", "sad", "neutral"])
logMel_train_encoded_labels = le.transform(logMel_train_labels)
logMel_valid_encoded_labels = le.transform(logMel_valid_labels)

# to decode these labels use the following command: example
#results = list(le.inverse_transform([2, 2, 1]))

In [3]:
# 1.3.1 One hot encoding for the target vector
from keras.utils import np_utils
y_logMel_train_cat = np_utils.to_categorical(logMel_train_encoded_labels)
y_logMel_valid_cat = np_utils.to_categorical(logMel_valid_encoded_labels)


In [4]:
# 1.4 reshape data into 4D matrix (CNN takes 4D) 
# params
input_channels = 1
#X_train_4D = np.expand_dims(logMel_train_features, axis=-1)
X_logMel_train_4D = logMel_train_features.reshape(logMel_train_features.shape[0], input_height, 
                                                  input_width , input_channels).astype('float32')
print(X_logMel_train_4D.shape)
X_logMel_valid_4D = logMel_valid_features.reshape(logMel_valid_features.shape[0], input_height, 
                                                  input_width , input_channels).astype('float32')
X_logMel_test_4D = logMel_test_features.reshape(logMel_test_features.shape[0], input_height, 
                                                  input_width , input_channels).astype('float32')

(5531, 750, 26, 1)


### Creating CNN Model

In [5]:
# Step 2: define model
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers.merge import Concatenate
from keras import optimizers

# Paramters
filters_l1 = 100
filter_heights = [5,10]
filter_width = input_width
pool_height = 30
pool_width = 1
pool_stride = 3
filter_stride = 3
num_classes = 4
loss_func = 'categorical_crossentropy' #'mse'
#MSE is used for regression mostly (see ex3 and https://www.youtube.com/watch?v=IVVVjBSk9N0)
optimizer_func = 'adam'
conv_actv = 'relu'
dropout_rate = 0.5
img_path='network_image.png'

#2.0 begin model design
# Parallel layers code from KEras.pdf and here:
# https://stackoverflow.com/questions/43151775/how-to-have-parallel-convolutional-layers-in-keras
#2.1 input shape defined for our variant filter-width layer 
model_input = Input(shape=X_logMel_train_4D.shape[1:])
conv_blocks = []
for i in range(0,2):
    # add a conv layer with the specified width 
    conv = Conv2D(filters=filters_l1, kernel_size=(filter_heights[i], filter_width), 
                  activation=conv_actv,strides=filter_stride)(model_input)
    # add pooling 
    pooled = MaxPooling2D(pool_size=(pool_height, pool_width),strides =pool_stride)(conv)
    # flatten results
    pooled_flat = Flatten()(pooled)
    conv_blocks.append(pooled_flat)
if len(conv_blocks) > 1: # here we merge
    cp_all = Concatenate()(conv_blocks)
else:
    cp_all = conv_blocks[0]   
# 2.2 add dense layer to handle the output of the previous layer
model_output = Dense(100, activation='relu')(cp_all)

#2.3 define sub model (our convoluaitonal layer with various filter sizes)
sub_model = Model(model_input, model_output)

#2.4 define overall model
model = Sequential()
# 2.5 first convolutional layer
model.add(sub_model)
BatchNormalization(momentum=0.99)
#2.6 add dropout
model.add(Dropout(dropout_rate))
BatchNormalization(momentum=0.99)
#2.7 add dense (fully connected) layer for the softmax
model.add(Dense(num_classes, activation='softmax'))  
# note from slides: # params >> # training instances is bad
model.summary()
# 2.8 save structure of model
from keras.utils import plot_model
plot_model(model, to_file=img_path)

#2.9 compile model
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=True)
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=False)
optimizer_func = adam #sgd 
model.compile(loss=loss_func, optimizer=optimizer_func, metrics=['accuracy'])          

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 100)               1509300   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 1,509,704
Trainable params: 1,509,704
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Step 3: train model

#3.0 use earlstopping to avoid overfitting with epochs
from keras.callbacks import ModelCheckpoint, EarlyStopping
# define path to save model
model_path = './parallel_cnn_BN.h5'
# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]
# Paramters
total_epochs = 100
batch_s = 32
number_training_samples = X_logMel_train_4D.shape[1]

#history = model.fit(X_logMel_train_4D, y_logMel_train_cat, validation_data=(X_logMel_valid_4D, y_logMel_valid_cat), epochs=total_epochs, batch_size=batch_s, verbose=2, shuffle=True)
history = model.fit(X_logMel_train_4D, y_logMel_train_cat, validation_data=(X_logMel_train_4D, y_logMel_train_cat), epochs=total_epochs, batch_size=batch_s, verbose=2, shuffle=True)
        
# 3.1 evaluate model on dev set
#score = model.evaluate(X_logMel_valid_4D, y_logMel_valid_cat, verbose=0)
score = model.evaluate(X_logMel_train_4D, y_logMel_train_cat, verbose=0)
#3.2 get score
print("Accuracy on dev set:{0}%".format(score[1]*100))

Train on 5531 samples, validate on 5531 samples
Epoch 1/100
 - 6s - loss: 1.2493 - acc: 0.3918 - val_loss: 1.1269 - val_acc: 0.4826
Epoch 2/100
 - 6s - loss: 1.1843 - acc: 0.4325 - val_loss: 1.0867 - val_acc: 0.5039
Epoch 3/100
 - 5s - loss: 1.1578 - acc: 0.4493 - val_loss: 1.0700 - val_acc: 0.5005
Epoch 4/100
 - 5s - loss: 1.1356 - acc: 0.4654 - val_loss: 1.0365 - val_acc: 0.5343
Epoch 5/100
 - 5s - loss: 1.1123 - acc: 0.4739 - val_loss: 1.0322 - val_acc: 0.5406
Epoch 6/100
 - 5s - loss: 1.1048 - acc: 0.4867 - val_loss: 1.0019 - val_acc: 0.5509
Epoch 7/100
 - 5s - loss: 1.0845 - acc: 0.4994 - val_loss: 0.9855 - val_acc: 0.5766
Epoch 8/100
 - 5s - loss: 1.0709 - acc: 0.4932 - val_loss: 0.9732 - val_acc: 0.5672
Epoch 9/100
 - 5s - loss: 1.0490 - acc: 0.5149 - val_loss: 0.9662 - val_acc: 0.5946
Epoch 10/100
 - 5s - loss: 1.0386 - acc: 0.5259 - val_loss: 0.9437 - val_acc: 0.5894
Epoch 11/100
 - 5s - loss: 1.0285 - acc: 0.5265 - val_loss: 0.9232 - val_acc: 0.6138
Epoch 12/100
 - 5s - loss:

Epoch 97/100
 - 5s - loss: 0.3315 - acc: 0.8715 - val_loss: 0.1751 - val_acc: 0.9604
Epoch 98/100
 - 5s - loss: 0.3309 - acc: 0.8660 - val_loss: 0.1605 - val_acc: 0.9664
Epoch 99/100
 - 5s - loss: 0.3224 - acc: 0.8801 - val_loss: 0.1715 - val_acc: 0.9649
Epoch 100/100
 - 5s - loss: 0.3284 - acc: 0.8678 - val_loss: 0.1661 - val_acc: 0.9689
Accuracy on dev set:96.8902549192%


### Training History Visualization

In [7]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()